# Convolutional Neural Networks
The contents of this notebook were lifted from __"Deep Learning With Python"__

What we are going to cover in this notebook:
-  Understanding convolutional neural networks (convnets)
-  Using __data augmentation__ to mitigate __overfitting__
-  Using a __pretrained convnet__ to do __feature extraction__
-  __Fine-tuning__ a pretrained convnet

Convolutional neural networks, also known as __convnets__, a type of deep-learning model almost universally used in computer vision applications.

## Introduction to convnets
Before diving into the theory, lets look at the __MNIST__ dataset to understand image data representation.

### MNIST dataset
A classic in the machine-learning community.<br>
The problem we’re trying to solve here is to classify __grayscale images__ of __handwritten
digits (28 × 28 pixels) into their 10 categories (0 through 9)__. <br>
The dataset contains __60,000__ training images, plus __10,000__ test images.

![](figs/fig1.png)

#### 2-D image of Greyscale
![](figs/fig2.png)

#### 4-D Tensor of Image data
![](figs/fig3.png)

### Instantiating a small convnet

Before we do a deep dive into the theory of convnets, lets do a quick example to see how convnets are built in keras

In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Importantly, a convnet takes as input tensors of shape __(image_height, image_width, image_channels)__ (not including the batch dimension)

Let’s display the architecture of the convnet so far:

In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### NOTE:
-  The output of every __Conv2D__ and __MaxPooling2D__ layer is a __3D tensor__ of shape (__height, width, channels__).
-  The __width__ and __height__ dimensions tend to shrink as you go deeper in the network. 
-  The __number__ of channels is controlled by the first argument passed to the Conv2D layers (32 or 64)

### Adding a classifier on top of the convnet

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Compiling the convnet

In [4]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

### Preparing the data

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [7]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

### Training the Model

In [8]:
model.fit(train_images, train_labels, epochs=3, batch_size=64)

Epoch 1/3
60000/60000 [==============================] - 69s - loss: 0.1673 - acc: 0.9479    
Epoch 2/3
60000/60000 [==============================] - 68s - loss: 0.0457 - acc: 0.9854    
Epoch 3/3
60000/60000 [==============================] - 65s - loss: 0.0324 - acc: 0.9900    


### Evaluate Model 

In [9]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

 9952/10000 [============================>.] - ETA: 0s

0.9893

### Compare Performance to Dense Network

In [16]:
err_dense = 1 - 0.9757
err_conv1 = 1 - 0.9893
print("% Decrease in Error rate = ", 100 * ((err_dense - err_conv1) / err_dense))

% Decrease in Error rate =  55.96707818930021


#### Question:
But why does this simple convnet work so well, compared to a densely connected
model?<br>
To answer this, let’s dive into what the __*Conv2D*__ and __*MaxPooling2D*__ layers do

## The convolution operation
The fundamental difference between a __densely connected layer__ and a __convolution
layer__ is this: <br>"__Dense layers learn global patterns in their input feature space, whereas convolution layers learn local patterns__"

![](figs/fig4.png)

Images can be broken into local patterns like __edges, textures__ etc

-  The patterns they learn are __translation invariant__. After learning a certain pattern in the lower-right corner of a picture, a convnet can recognize it anywhere.
-  They can learn __spatial hierarchies of patterns__ (see figure below). 
    -  a first convolution layer will learn small local patterns such as edges, 
    -  a second convolution layer will learn larger patterns made of the features of the first layers, and so on. 
-  This allows convnets to efficiently learn increasingly complex and abstract visual concepts (because the visual world is fundamentally spatially hierarchical).

![](figs/fig5.png)

Convolutions operate over 3D tensors, called __feature maps__, with __two spatial axes__ (height
and width) as well as a __depth axis__ (also called the __channels axis__). 

For an __RGB image__, the dimension of the __depth axis is 3__, because the image has __three color channels: red,
green, and blue__. <br>
For a __black-and-white picture__, like the MNIST digits, the depth is 1 (__levels of gray__)

### Key parameters of convolution
-  __Size of the patches__ extracted from the inputs: these are typically __3 × 3__ or __5 × 5__.
-  __Depth of the output feature map__ The number of filters computed by the convolution.

The example started with a depth of 32 and ended with a depth of 64.

Size of the patches extracted from the inputs—These are typically 3 × 3 or 5 × 5. In the
example, they were 3 × 3, which is a common choice.
 Depth of the output feature map—The number of filters computed by the convolution.
The example started with a depth of 32 and ended with a depth of 64.

In Keras __Conv2D layers__, these parameters are the first arguments passed to the layer: __*Conv2D(output_depth, (window_height, window_width))*__.

### How it works
A convolution works by __sliding these windows__ of size __3 × 3 or 5 × 5__ over the 3D input
__feature map__, stopping at every possible location, and extracting the 3D patch of surrounding
features __(shape (window_height, window_width, input_depth))__. 

Each such __3D patch is then transformed__ (via a tensor product with the same learned weight
matrix, called the __convolution kernel__) into a __1D vector__ of shape (output_depth,)

![](figs/fig6_1.png)

### Padding and Strides

Note that the output width and height may differ from the input width and height.
They may differ for two reasons:
-  Border effects, which can be countered by padding the input feature map
-  The use of strides - the distance between __center tiles__ of two successive windows i.e. __input patch centers__

In __Conv2D__ layers, padding is configurable via the padding argument, which takes two
values:
-  __"valid"__, which means __no padding__ (only valid window locations will be used)
-  __"same"__, which means __“pad in such a way as to have an output with the same width
and height as the input.__” The padding argument defaults to "valid".

### The max-pooling operation
Max pooling aggressively downsample feature maps.<br>
Max pooling consists of __extracting windows__ from the input feature maps and outputting
the __max value of each channel__.

#### Why Maxpool?
The main purpose of downsampling is to reduce the number of __feature-map coefficients__ to process, as well as to induce __spatial-filter hierarchies__ by making successive convolution layers look at __increasingly large windows__

#### A Resonable Subsampling strategy
The most reasonable subsampling strategy is to first produce __dense maps of features (via unstrided convolutions)__ and then look at the __maximal activation__ of the features over small patches